# Introduction

### Project initialization and setup

Importing all of the libraries that will be used. In the project.

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

Connecting the DB to SQL

In [ ]:
DB_PATH = "viewer_interactions.db"

try:
    conn = sqlite3.connect(DB_PATH)
    print("Connected successfully!")
except sqlite3.Error as e:
    print("Connection failed:", e)

In [ ]:
# Display options
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

Listing all the tables

In [ ]:
tables_query = """
               SELECT name
               FROM sqlite_master
               WHERE type='table'
               ORDER BY name; \
               """

tables_df = pd.read_sql_query(tables_query, conn)
print("Tables in the database:")
display(tables_df)

Creating a dictionary of type table_name -> DataFrame

In [ ]:
table_names = tables_df["name"].tolist()

schemas = {}

for table in table_names:
    pragma_query = f"PRAGMA table_info({table});"
    schema_df = pd.read_sql_query(pragma_query, conn)
    schemas[table] = schema_df
    print(f"\nSchema for table '{table}':")
    display(schema_df)

In [ ]:
# creating a dictionary to map dataframes
DF_MAP = {}

### Methods

Add new data frame function

In [ ]:
def add_new_df(global_name: str, dfs_key: str, dataframe=None):

    # if dataframe already exists in dfs → ignore "dataframe" input
    if dfs_key in dfs:
        globals()[global_name] = dfs[dfs_key]
        DF_MAP[global_name] = dfs_key
        print(f"Mapped existing dfs['{dfs_key}'] to global '{global_name}'")
        return

    # new dataframe
    if dataframe is None:
        raise ValueError(
            f"dfs_key '{dfs_key}' does not exist in dfs and no dataframe was provided."
        )

    dfs[dfs_key] = dataframe
    globals()[global_name] = dataframe
    DF_MAP[global_name] = dfs_key

    print(f"Added NEW df: global '{global_name}' → dfs['{dfs_key}']")

Syn data frame function

In [ ]:
def sync_dataframe(*dfs_to_refresh):
    globals_dict = globals()

    for df_obj in dfs_to_refresh:
        updated = False

        for global_name, dfs_key in DF_MAP.items():
            if df_obj is globals_dict[global_name]:
                globals_dict[global_name] = dfs[dfs_key]
                updated = True
                break

        if not updated:
            raise ValueError("Unknown dataframe passed!")

Searching function

In [ ]:
# takes the name of the table, the name of the target key, and the value as inputs. Returns singular or multiple dataframes based on the request.

# can also search for null values if value is set to None.

def search_by_parameter(table_name, key, value):
    df = dfs[table_name]

    if value is None:
        return df[df[key].isna()]

    return df[df[key] == value]

In [ ]:
# calculates std deviation
# make sure no nulls are included in the calculation!

def manual_std(values):
    arr = np.array(values, dtype=float)

    n = len(arr)

    if n <= 1:
        return 0.0

    mean = arr.mean()
    return np.sqrt(((arr - mean) ** 2).mean())

In [ ]:
def covert_string_to_date(date_str):
    if date_str is None or pd.isna(date_str):
        return None
    try:
        return datetime.strptime(date_str, "%Y-%m-%d")
    except:
        return None

In [ ]:
def elbow_method(X, k_min=1, k_max=10, random_state=42, plot=True):
    """
    Считает значения inertia для разных k (кол-во кластеров)
    и, при желании, рисует elbow-график.

    X      - матрица признаков (например, X_scaled)
    k_min  - минимальное число кластеров
    k_max  - максимальное число кластеров
    plot   - рисовать ли график
    """

    results = []

    for k in range(k_min, k_max + 1):
        kmeans = KMeans(
            n_clusters=k,
            random_state=random_state,
            n_init=10
        )
        kmeans.fit(X)
        results.append({"k": k, "inertia": kmeans.inertia_})

    elbow_df = pd.DataFrame(results)

    if plot:
        plt.figure()
        plt.plot(elbow_df["k"], elbow_df["inertia"], marker="o")
        plt.xlabel("Number of clusters (k)")
        plt.ylabel("Inertia")
        plt.title("Elbow method")
        plt.xticks(elbow_df["k"])
        plt.show()

    return elbow_df

In [ ]:
def compute_silhouette_score(X, labels, sample_size=None, random_state=42, metric="euclidean"):
    """
    X           : feature matrix (preferably after StandardScaler)
    labels      : cluster labels (k-means, agglomerative, etc.)
    sample_size: if None, we consider all objects, otherwise we take a random subsample of the specified size
    random_state: random state fixation for reproducibility
    metric: distance metric (Euclidean by default)
    """
    X = np.asarray(X)
    labels = np.asarray(labels)

    if X.shape[0] != labels.shape[0]:
        raise ValueError(f"X has {X.shape[0]} rows, but labels has {labels.shape[0]} elements")

    if sample_size is not None and sample_size < len(X):
        rng = np.random.RandomState(random_state)
        idx = rng.choice(len(X), size=sample_size, replace=False)
        X_used = X[idx]
        labels_used = labels[idx]
    else:
        X_used = X
        labels_used = labels

    score = silhouette_score(X_used, labels_used, metric=metric)
    return score

In [ ]:
def inspect_cluster(movies_df, merged_df, cluster_id):
    """
    movies_df — table with movie_id and cluster_id 
    merged_df — merged table with movie_id + title
    cluster_id — cluster index to inspect
    """
    cluster = movies_df[movies_df["cluster_id"] == cluster_id]

    display(cluster.describe())

    detailed = cluster.merge(
        merged_df[["movie_id", "title"]].drop_duplicates(),
        on="movie_id",
        how="left"
    )

    print(f"\nФильмы в кластере {cluster_id}:")
    return detailed

### Database initialization

implementing dfs and importing data

In [ ]:
dfs = {} # dictionary that maps the name of the table to the related data frame
# isnt it simpler if we just use universal dataframes for each dataset instead of having to worry about dictionaries

for t in table_names:
    df = pd.read_sql_query(f"SELECT * FROM {t};", conn)
    dfs[t] = df
    print(f"\nLoaded table '{t}' with shape {df.shape}")

In [ ]:
# adding our base data frames
add_new_df("movies_stats", "movie_statistics", dfs["movie_statistics"])
add_new_df("movies",       "movies",           dfs["movies"])
add_new_df("user_stats",   "user_statistics",  dfs["user_statistics"])
add_new_df("viewer_ratings", "viewer_ratings", dfs["viewer_ratings"])

### Diagnostics

Counting all missing values, diagnostics purposes only

In [ ]:
for name, df in dfs.items():
    print(f"\n{name} missing values (%):")
    missing_pct = df.isna().mean() * 100
    display(missing_pct.to_frame("missing_%"))

In [ ]:
missing_after = user_stats[user_stats["std_rating"].isna()].copy()

print("Всего пользователей с NaN std:", len(missing_after))

# 1) Есть ли они вообще в viewer_ratings?
print(
    "Сколько из них встречаются в viewer_ratings:",
    missing_after["customer_id"].isin(viewer_ratings["customer_id"]).sum()
)

### Movie Statistics Calculations

A function to calculate missing std. ratings of films

In [ ]:
# Синхронизируем dataframes
sync_dataframe(movies_stats, viewer_ratings)

# Убираем NaN-рейтинги и считаем std для фильмов
viewer_ratings_clean = viewer_ratings[~viewer_ratings["rating"].isna()]

movie_std = (
    viewer_ratings[~viewer_ratings["rating"].isna()]
    .groupby("movie_id")["rating"]
    .apply(list)
    .reset_index(name="ratings")
)

# manual_std уже объявлена выше и игнорирует NaN (или можно явно указать ignore_nan=True)
movie_std["std_rating"] = movie_std["ratings"].apply(manual_std)
movie_std = movie_std[["movie_id", "std_rating"]]

# Старый процент пропусков
old_null_pct = dfs["movie_statistics"]["std_rating"].isna().mean() * 100

# Мержим новые значения std по movie_id
movies_stats = movies_stats.merge(
    movie_std,
    on="movie_id",
    how="left",
    suffixes=("", "_new")
)

# Обновляем std_rating
movies_stats["std_rating"] = movies_stats["std_rating_new"]
movies_stats.drop(columns=["std_rating_new"], inplace=True)

# Сохраняем обратно
dfs["movie_statistics"] = movies_stats

# Новый процент пропусков
new_null_pct = movies_stats["std_rating"].isna().mean() * 100

# Улучшения
improvement_abs = old_null_pct - new_null_pct
improvement_rel = (improvement_abs / old_null_pct) * 100 if old_null_pct > 0 else 0

print(f"Missing values reduced from {old_null_pct:.2f}% to {new_null_pct:.2f}%.")
print(f"Absolute improvement: {improvement_abs:.2f}%")
print(f"Relative improvement: {improvement_rel:.2f}% better than before.")

In [ ]:
#Figure out how to drop na values in general
movies_stats = dfs['movie_statistics']

print(f"Before cleaning: {len(movies_stats)} movies")
#movies_stats = movies_stats.dropna(subset=['std_rating'])
#dfs['movie_statistics'] = movies_stats
print(f"After removing single-rating movies: {len(movies_stats)} movies")

Calculating the missing total_ratings of movies

In [ ]:
sync_dataframe(movies_stats)

# Collecting all the movies with absent total_rating in a dictionary
missing_dict = {}

missing = search_by_parameter('movie_statistics', 'total_ratings', None)
missing_dict = {row.movie_id: 0 for row in missing.itertuples(index=False)}

# Iterating through viewer_ratings and manually counting the ratings for each film
for row in viewer_ratings.itertuples(index=False):
    movie_id = row.movie_id
    if movie_id in missing_dict:
        missing_dict[movie_id] += 1

# Update movie_stats
for row in movies_stats.itertuples(index=True):
    if row.movie_id in missing_dict:
        movies_stats.at[row.Index, "total_ratings"] = missing_dict[row.movie_id]

dfs["movie_statistics"] = movies_stats

Calculating Missing Averages

In [ ]:
sync_dataframe(movies_stats, viewer_ratings)

# Finding movies with null avg_rating
missing_avg = search_by_parameter('movie_statistics', 'avg_rating', None)

# Creating a dict of type { movie_id : avg_rating }
# set 0 as base value for now, might change it later
missing_avg_dict = {row.movie_id: 0 for row in missing_avg.itertuples(index=False)}

# Storing the sum of all ratings for each movie
rating_sums = {movie_id: 0 for movie_id in missing_avg_dict}

# Iterating through viewer ratings and adding to sum if movie_id matches
for row in viewer_ratings.itertuples(index=False):
    movie_id = row.movie_id
    rating = row.rating

    if movie_id in missing_avg_dict:
        rating_sums[movie_id] += rating

for row in movies_stats.itertuples():
    movie_id = row.movie_id

    if movie_id in rating_sums:
        total = row.total_ratings # I'm assuming that my calculations of total_ratings per movie is correct ang i got rid of                            all null values

        # IF FORE SOME MAGICAL REASON THERE IS STILL A NULL THEN IGNORE
        if pd.isna(total) or total == 0:
            avg = 0
        else:
            avg = rating_sums[movie_id] / total

        movies_stats.at[row.Index, "avg_rating"] = avg

Calculating the missing min and max ratings for movies

In [ ]:
sync_dataframe(movies_stats)

# Find movies with missing min_rating and max_rating using your function
missing_min = search_by_parameter('movie_statistics', 'min_rating', None)
missing_max = search_by_parameter('movie_statistics', 'max_rating', None)

# Combine them as some movies may be in both
missing_ids = set(missing_min["movie_id"]) | set(missing_max["movie_id"])

# Take only ratings for the movies we care
relevant_ratings = viewer_ratings[viewer_ratings["movie_id"].isin(missing_ids)]

# Building a nested dict {movie_id : {"min": ..., "max": ...}}
min_max_dict = {}

for row in relevant_ratings.itertuples(index=False):
    movie_id = row.movie_id
    rating = row.rating

    if movie_id not in min_max_dict:
        min_max_dict[movie_id] = {"min": rating, "max": rating}
    else:
        if rating < min_max_dict[movie_id]["min"]:
            min_max_dict[movie_id]["min"] = rating
        if rating > min_max_dict[movie_id]["max"]:
            min_max_dict[movie_id]["max"] = rating

# Update movie_statistics
for row in movies_stats.itertuples(index=True):
    movie_id = row.movie_id

    if movie_id in min_max_dict:
        if pd.isna(row.min_rating):
            movies_stats.at[row.Index, "min_rating"] = min_max_dict[movie_id]["min"]
        if pd.isna(row.max_rating):
            movies_stats.at[row.Index, "max_rating"] = min_max_dict[movie_id]["max"]

dfs["movie_statistics"] = movies_stats

Finding missing unique users

In [ ]:
sync_dataframe(movies_stats)

missing_unique = search_by_parameter('movie_statistics', 'unique_users', None)

# creating my favorite movie set
missing_movie_ids = {row.movie_id for row in missing_unique.itertuples(index=False)}

# creating a dict movie_id: customer_id
unique_users_dict = {movie_id: set() for movie_id in missing_movie_ids}

# Gathering unique users
for row in viewer_ratings.itertuples(index=False):
    movie_id = row.movie_id

    # as always im getting only those movies which have null for unique users
    if movie_id in unique_users_dict:
        unique_users_dict[movie_id].add(row.customer_id)

# Counting unique users
updated = 0
for movie_id, users in unique_users_dict.items():
    count = len(users)  # unique users count

    movies_stats.loc[
        movies_stats["movie_id"] == movie_id,
        "unique_users"
    ] = count

    updated += 1

# just in case if a movie has 0 ratings im setting unique users to 0
movies_stats["unique_users"] = movies_stats["unique_users"].fillna(0).astype(int)

# Updating
dfs["movie_statistics"] = movies_stats

Checking for any duplicate movie ids and removing them from all datasets to clean the data.

### User_statistics calculations

Calculating user's std rating

In [ ]:
sync_dataframe(user_stats)

# grouping by user id
user_std = (
    viewer_ratings[~viewer_ratings["rating"].isna()]
    .groupby("customer_id")["rating"]
    .apply(list)
    .reset_index(name="ratings")
)

user_std["std_rating"] = user_std["ratings"].apply(manual_std)
user_std = user_std[["customer_id", "std_rating"]]

# old percentage
old_null_pct = user_stats["std_rating"].isna().mean() * 100

# merging
user_stats = user_stats.merge(
    user_std,
    on="customer_id",
    how="left",
    suffixes=("", "_new")
)

# replacing old std with new
user_stats["std_rating"] = user_stats["std_rating_new"]
user_stats.drop(columns=["std_rating_new"], inplace=True)

# updating
dfs["user_statistics"] = user_stats

# calculating new percantages
new_null_pct = user_stats["std_rating"].isna().mean() * 100

# calculating improvements
improvement_abs = old_null_pct - new_null_pct
improvement_rel = (improvement_abs / old_null_pct) * 100 if old_null_pct > 0 else 0

print(f"Missing values reduced from {old_null_pct:.2f}% to {new_null_pct:.2f}%.")
print(f"Absolute improvement: {improvement_abs:.2f}%")
print(f"Relative improvement: {improvement_rel:.2f}% better than before.")

Calculating user's total ratings

In [ ]:
sync_dataframe(user_stats)

# Collecting all the users with absent total_ratings
missing_dict = {}

missing = search_by_parameter('user_statistics', 'total_ratings', None)
missing_dict = {row.customer_id: 0 for row in missing.itertuples(index=False)}

# Iterating through viewer_ratings and manually counting the ratings for each user
for row in viewer_ratings.itertuples(index=False):
    if pd.isna(row.rating):
        continue

    customer_id = row.customer_id
    if customer_id in missing_dict:
        missing_dict[customer_id] += 1

# Update user_stats
for row in user_stats.itertuples(index=True):
    if row.customer_id in missing_dict:
        user_stats.at[row.Index, "total_ratings"] = missing_dict[row.customer_id]

dfs["user_statistics"] = user_stats

Calculating user's avg rating

In [ ]:
sync_dataframe(user_stats)

# Finding all users with null avg_rating
missing_avg = search_by_parameter('user_statistics', 'avg_rating', None)

# Creating a dict of type { customer_id : avg_rating }
# set 0 as base value
missing_avg_dict = {row.customer_id: 0 for row in missing_avg.itertuples(index=False)}

# Storing the sum of all ratings for each movie
rating_sums = {customer_id: 0 for customer_id in missing_avg_dict}

# Iterating through viewer ratings and adding to sum if movie_id matches
for row in viewer_ratings.itertuples(index=False):
    if pd.isna(row.rating):
        continue

    customer_id = row.customer_id
    rating = row.rating

    if customer_id in missing_avg_dict:
        rating_sums[customer_id] += rating

for row in user_stats.itertuples():
    customer_id = row.customer_id

    if customer_id in rating_sums:
        total = row.total_ratings

        if pd.isna(total) or total == 0:
            avg = 0
        else:
            avg = rating_sums[customer_id] / total

        user_stats.at[row.Index, "avg_rating"] = avg

In [ ]:
#activity days
sync_dataframe(user_stats)

missing = search_by_parameter('user_statistics', 'activity_days', None)

# creating a dict
missing_dict = {row.customer_id: 0 for row in missing.itertuples(index=False)}

for row in user_stats.itertuples():
    customer_id = row.customer_id

    if customer_id not in missing_dict: # wroking only with users with missing activity_days
        continue

    first_date = covert_string_to_date(row.first_rating_date)
    last_date = covert_string_to_date(row.last_rating_date)

    # null exception
    if first_date is None or last_date is None:
        continue

    # calcing the difference
    difference = (last_date - last_date).days

    # updating activity_days
    user_stats.at[row.Index, 'activity_days'] = difference

null_percentage = user_stats['activity_days'].isna().mean() * 100
print(f"Missing activity_days: {null_percentage}%")

In [ ]:
#minmax
sync_dataframe(user_stats)

# Find users with missing min_rating and max_rating using your function
missing_min = search_by_parameter('user_statistics', 'min_rating', None)
missing_max = search_by_parameter('user_statistics', 'max_rating', None)

# Combine them as some users may be in both
missing_ids = set(missing_min["customer_id"]) | set(missing_max["customer_id"])

# Init dictionary for users we need to compute
missing_users = {
    customer_id: {"min": -1, "max": -1} #setting base to -1
    for customer_id in missing_ids
}

for row in viewer_ratings.itertuples(index=False):
    customer_id = row.customer_id
    rating = row.rating

    if customer_id in missing_users:
        # updating min
        if missing_users[customer_id]["min"] == -1 or rating < missing_users[customer_id]["min"]:
            missing_users[customer_id]["min"] = rating

        # updating max
        if missing_users[customer_id]["max"] == -1 or rating > missing_users[customer_id]["max"]:
            missing_users[customer_id]["max"] = rating

# Updating user stats
for row in user_stats.itertuples():
    customer_id = row.customer_id

    if customer_id in missing_users:
        min_val = missing_users[customer_id]["min"]
        max_val = missing_users[customer_id]["max"]

        # if the guy hasn't rated anything im making sure we set min or max to none so we get rid of them in the future
        if min_val == -1 and max_val == -1:
            min_val = None
            max_val = None

        user_stats.at[row.Index, "min_rating"] = min_val
        user_stats.at[row.Index, "max_rating"] = max_val

dfs["user_statistics"] = user_stats

In [ ]:
sync_dataframe(user_stats)
missing_unique = search_by_parameter('user_statistics', 'unique_movies', None)

missing_ids = set(missing_unique["customer_id"])

unique_movies_dict = {customer_id: set() for customer_id in missing_ids}

for row in viewer_ratings.itertuples(index=False):
    user_id = row.customer_id
    movie_id = row.movie_id

    if user_id in unique_movies_dict:
        unique_movies_dict[user_id].add(movie_id)

for row in user_stats.itertuples():
    user_id = row.customer_id

    if user_id in unique_movies_dict:
        count_unique = len(unique_movies_dict[user_id])

        if count_unique == 0:
            value = None
        else:
            value = count_unique

        user_stats.at[row.Index, "unique_movies"] = value

# updating
dfs["user_statistics"] = user_stats

### Merging datasets

Merging the Movies and movie statis filling in missing values on either dataset and converting all of the dates to type DateTime as well as all counts and years to integers.
This is in order to clean our movie data before merging it with our user data to fill in any recoverable missing values.
Rows with missing values will be dropped.

In [ ]:
movies = dfs['movies'].copy() # does this overwrite the movies we have right now? maybe call it something else.
movie_stats = dfs['movie_statistics'].copy()
viewer_ratings = dfs['viewer_ratings'].copy()

bad_ids_movies = movies.loc[movies['movie_id'].duplicated(keep=False), 'movie_id'].unique().tolist()
bad_ids_stats = movie_stats.loc[movie_stats['movie_id'].duplicated(keep=False), 'movie_id'].unique().tolist()
bad_ids = list(set(bad_ids_movies + bad_ids_stats))
print('bad movie ids to remove: ', bad_ids)

viewer_ratings = viewer_ratings[~viewer_ratings['movie_id'].isin(bad_ids)]
movies = movies[~movies['movie_id'].isin(bad_ids)]
movie_stats = movie_stats[~movie_stats['movie_id'].isin(bad_ids)]

movies['year_of_release'] = (pd.to_numeric(movies['year_of_release'], errors='coerce').astype('Int64'))
movie_stats['total_ratings'] = (pd.to_numeric(movie_stats['total_ratings'], errors='coerce').astype('Int64'))
movie_stats['unique_users'] = (pd.to_numeric(movie_stats['unique_users'], errors='coerce').astype('Int64'))
movie_stats['year_of_release'] = (pd.to_numeric(movie_stats['year_of_release'], errors='coerce').astype('Int64'))

movie_stats['first_rating_date'] = pd.to_datetime(movie_stats['first_rating_date'], errors='coerce')
movie_stats['last_rating_date'] = pd.to_datetime(movie_stats['last_rating_date'], errors='coerce')

# this will automatically remove rows with missing data
movie_full = movies.merge(movie_stats, on='movie_id', how='inner', suffixes=('_movies', '_stats'))

movie_full['title'] = movie_full['title_movies'].combine_first(movie_full['title_stats'])
movie_full['year_of_release'] = movie_full['year_of_release_movies'].combine_first(movie_full['year_of_release_stats'])

movie_full = movie_full.drop(columns=['title_movies', 'title_stats', 'year_of_release_movies', 'year_of_release_stats'])

dfs['movies'] = movies
dfs['movie_statistics'] = movie_stats
dfs['viewer_ratings'] = viewer_ratings
add_new_df('movie_full', 'movie_full', movie_full)

In [ ]:
print("movies dtypes:")
print(movies.dtypes)

print("\nmovie_stats dtypes:")
print(movie_stats.dtypes)

print("\nmovie_full dtypes:")
print(movie_full.dtypes)

- Converts the date parameter in viewer_ratings to datetime.
- Merges viewer_ratings, movies, movie_statistics and user_statistics into one dataset as merged_data.

In [ ]:
movie_full = dfs['movie_full'].copy()
user_stats = dfs['user_statistics'].copy()
viewer_ratings = dfs['viewer_ratings'].copy()


viewer_ratings['date'] = pd.to_datetime(viewer_ratings['date'], errors = 'coerce')
user_stats['first_rating_date'] = pd.to_datetime(user_stats['first_rating_date'], errors = 'coerce')
user_stats['last_rating_date'] = pd.to_datetime(user_stats['last_rating_date'], errors = 'coerce')

user_stats['total_ratings'] = (pd.to_numeric(user_stats['total_ratings'], errors='coerce').astype('Int64'))
user_stats['unique_movies'] = (pd.to_numeric(user_stats['unique_movies'], errors='coerce').astype('Int64'))
user_stats['activity_days'] = (pd.to_numeric(user_stats['activity_days'], errors='coerce').astype('Int64'))


# create user and movie specific columns
movie_full = movie_full.rename(columns={
    'total_ratings':     'movie_total_ratings',
    'avg_rating':        'movie_avg_rating',
    'std_rating':        'movie_std_rating',
    'min_rating':        'movie_min_rating',
    'max_rating':        'movie_max_rating',
    'first_rating_date': 'movie_first_rating_date',
    'last_rating_date':  'movie_last_rating_date'
})


user_stats = user_stats.rename(columns={
    'total_ratings':     'user_total_ratings',
    'avg_rating':        'user_avg_rating',
    'std_rating':        'user_std_rating',
    'min_rating':        'user_min_rating',
    'max_rating':        'user_max_rating',
    'first_rating_date': 'user_first_rating_date',
    'last_rating_date':  'user_last_rating_date'
})

# drop anomalous date for user stats
if 'anomalous_date' in viewer_ratings.columns:
    viewer_ratings = viewer_ratings.drop(columns=['anomalous_date'])

# merge user stats and viewer ratings
viewer_ratings_with_stats = viewer_ratings.merge(user_stats, on = 'customer_id', how = 'inner')

merged_data = viewer_ratings_with_stats.merge(movie_full, on = 'movie_id', how = 'inner')

# this will drop all rows with any null
merged_data = merged_data.dropna(how = 'any')

add_new_df('merged_data', 'merged_data', merged_data)
dfs['movie_full'] = movie_full
dfs['user_statistics'] = user_stats
dfs['viewer_ratings'] = viewer_ratings

print(merged_data.dtypes)

### Plotting Statistics and Overall findings

Ryder

In [ ]:
import seaborn as sns


# Markdown cell above
"""
### Global Rating Distribution

There is a clear over positive bias. This can be due to the fact that this was done on a streaming platform where users are more likely to rate higher
"""

plt.figure(figsize=(11, 6)) # made it this size for a better fit in the read me
ax = sns.histplot(
    data=dfs['viewer_ratings'],
    x='rating',
    bins=5,
    discrete=True, #tells seaborn to treat x as integer values
    color="#0062ff",
    edgecolor='white',
    alpha=0.85,  #transparency
    linewidth=1.5
)

# Add exact counts on top of each bar
for rect in ax.patches:
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width()/2., 
        height + 200_000,                    # a bit above the bar
        f'{int(height):,}',                  # adds commas: 12,345,678
        ha='center', va='bottom', fontsize=12, fontweight='bold'
    )

# Clean y-axis
plt.ylabel('Number of Ratings (millions)', fontsize=12)
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x/1e6:.1f}M'))

plt.title('Distribution of Viewer Ratings\n(Strong Positive Bias)', 
          fontsize=15, pad=20)
plt.xlabel('Rating (1–5)', fontsize=12)
plt.xticks(range(0, 7))
plt.ylim(0, 2_000_000)
plt.show()

In [ ]:
"""
### User Average Rating vs. Rating Variability

Scatter from user_statistics: High avg with low std = consistent positive raters. 
Valuable for grouping users (e.g., strict critics vs. easy fans). Alpha for density.
"""

sync_dataframe(user_stats)

plt.figure(figsize=(10, 6))
sns.scatterplot(data=user_stats, x='avg_rating', y='std_rating', alpha=0.4, color='#0062ff', edgecolor=None)
plt.title('User Average Rating vs. Standard Deviation\n(Rating Consistency Patterns)', fontsize=15, pad=20)
plt.xlabel('Average Rating Given (1–5)', fontsize=12)
plt.ylabel('Std Deviation of Ratings', fontsize=12)
plt.xlim(0, 5)
plt.ylim(0, 4)  # Std typically low due to bias
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# PERFECT PENTAGON — FINAL WORKING VERSION (Dec 2025)
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.patches import PathPatch

# CHANGE ONLY THESE TWO VALUES
avg_rating_value = user_stats['avg_rating'].mean()   # ← your real global avg from user_statistics
std_rating_value = user_stats['std_rating'].mean()   # ← your real global std from user_statistics

# Convert to 0–1 scale
pull_to_five = (avg_rating_value - 1.0) / 4.0
consistency  = np.clip(1.0 - (std_rating_value / 1.8), 0.1, 1.0)

# Pentagon vertices (5 corners + first repeated to close)
center = np.array([0.5, 0.5])
radius = 0.45
angles = np.linspace(0, 2*np.pi, 5, endpoint=False)
verts = center + radius * np.column_stack([np.cos(angles), np.sin(angles)])
verts = np.vstack([verts, verts[0]])  # close the loop → shape (6, 2)

# Blue pentagon (pulled toward Rating 5 = bottom-right)
blue_scale = 0.85 * pull_to_five
blue_verts = center + blue_scale * (verts - center)

# Red pentagon (size = consistency)
red_scale = 0.92 * consistency
red_verts = center + red_scale * (verts - center)

# Correct Path codes: 6 vertices → 6 codes
codes = [Path.MOVETO] + [Path.LINETO]*4 + [Path.CLOSEPOLY]

# Plot
fig, ax = plt.subplots(figsize=(10, 10))

# Outer frame
frame_path = Path(verts, codes)
ax.add_patch(PathPatch(frame_path, facecolor='none', edgecolor='black', linewidth=3))

# Red pentagon (consistency)
red_path = Path(red_verts, codes)
ax.add_patch(PathPatch(red_path, facecolor="#000000", alpha=0.7, edgecolor='#aa0000', linewidth=4))

# Blue pentagon (average rating)
blue_path = Path(blue_verts, codes)
ax.add_patch(PathPatch(blue_path, facecolor="#00ffdd", alpha=0.75, edgecolor='#003380', linewidth=4))

# Corner labels
labels = ['1', '2', '3', '4', '5']
for (x, y), label in zip(verts[:-1], labels):
    ax.text(x, y, label, fontsize=16, fontweight='bold', ha='center', va='center',
            bbox=dict(boxstyle="round,pad=0.5", facecolor="white", alpha=0.95))

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal')
ax.axis('off')

ax.set_title(f"Global User Rating Personality Pentagon\n"
             f"Avg Rating = {avg_rating_value:.2f} | Std Dev = {std_rating_value:.2f}",
             fontsize=20, pad=40, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
'Plotting a timeline of ratings over time'

sync_dataframe(movies_stats)

plt.figure(figsize=(15, 10))
sns.scatterplot(data=movies_stats, x='year_of_release', y='avg_rating', alpha=0.4, color='#0062ff', edgecolor=None, s=35)
plt.title('Movie Rating VS. Year of Release\n(Trends Over Time)', fontsize=15, pad=20)
plt.xlabel('Year of Release', fontsize=12)
plt.ylabel('Average Rating Given (1–5)', fontsize=12)
plt.xlim(1890, 2010) #this included the data for all the movies in the data set as it goes from 1896 to 2005
plt.ylim(0, 7)  
#plt.scatter(x,y, s)
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

In [ ]:
def get_top_rated_movies(min_ratings=100, top_n=10):
    """
    Returns the top N best-rated movies with at least min_ratings.
    Merges movie_statistics and movies tables to get titles and years.
    """
    # Get movie stats and filter
    sync_dataframe(movies_stats)
    filtered = movies_stats[movies_stats['total_ratings'] >= min_ratings]
    
    # Sort by avg_rating descending
    top_movies = filtered.sort_values('avg_rating', ascending=False).head(top_n)
    
    # Select and reorder columns nicely
    result = top_movies[['movie_id', 'avg_rating', 'total_ratings']].copy()
    
    result = result.reset_index(drop=True)
    result.index = result.index + 1  # Start ranking at 1
    
    return result

# === USE IT LIKE THIS ===
print("TOP 10 BEST-RATED MOVIES (min 100 ratings):")
display(get_top_rated_movies(min_ratings=200, top_n=10))

# sync_dataframe(movies_stats)

# plt.figure(figsize=(15, 10))
# sns.scatterplot(data=get_top_rated_movies(min_ratings=100, top_n=10), x='movie_id', y='avg_rating', alpha=0.4, color='#0062ff', edgecolor=None, s=35)
# plt.title('top 10 best-rated movies (min 100 ratings)', fontsize=15, pad=20)
# plt.xlabel('Movies', fontsize=12)
# plt.ylabel('Average Rating Given (1–5)', fontsize=12)
# plt.xlim(str(search_by_parameter("movies", "movie_id", get_top_rated_movies(min_ratings=100, top_n=10)['movie_id'].min())["title"]), str(search_by_parameter("movies", "movie_id", get_top_rated_movies(min_ratings=100, top_n=10)['movie_id'].max())["title"])) #this included the data for all the movies in the data set as it goes from 1896 to 2005
# plt.ylim(0, 5)  
# #plt.scatter(x,y, s)
# plt.grid(True, linestyle='--', alpha=0.7)
# plt.show()

print(str(search_by_parameter("movies", "movie_id", get_top_rated_movies(min_ratings=200, top_n=10)['movie_id'].min())["title"]))

# ploting = []
# for ploting.len() in range(10):
#     ploting.append([search_by_parameter("movies", "movie_id", get_top_rated_movies(min_ratings=100, top_n=10)['movie_id'])])

# print("here"+ploting(1))

In [ ]:
def get_top_rated_movies(min_ratings=200, top_n=10):
    sync_dataframe(movies_stats)
    
    # Filter and sort — pure movie_statistics
    filtered = movies_stats[movies_stats['total_ratings'] >= min_ratings]
    top_movies = filtered.sort_values('avg_rating', ascending=False).head(top_n).copy()
    
    # Reset index to get ranking 1 to 10
    top_movies = top_movies.reset_index(drop=True)
    top_movies.index = top_movies.index + 1
    
    # Use your existing search function to get titles
    titles = []
    for movie_id in top_movies['movie_id']:
        result = search_by_parameter("movies", "movie_id", movie_id)
        title = result['title'].iloc[0] if not result.empty else f"Unknown ({movie_id})"
        year = int(result['year_of_release'].iloc[0]) if not result.empty and pd.notna(result['year_of_release'].iloc[0]) else ""
        titles.append(f"{title} ({year})" if year else title)
    
    top_movies['title_display'] = titles
    return top_movies

# === GET DATA ===
top10 = get_top_rated_movies(min_ratings=1000, top_n=10)

# === PLOT — TITLES ON X-AXIS (FIXED & WORKING) ===
plt.figure(figsize=(16, 9))

# Scatter plot with titles
sns.scatterplot(
    data=top10,
    x='title_display',
    y='avg_rating',
    s=150,
    color='#0062ff',
    edgecolor='navy',
    alpha=0.95,
    linewidth=3
)

# Add text above each point
for i, row in top10.iterrows():
    plt.text(
        x=i-1,  # categorical position
        y=row['avg_rating'] + 0.015,
        s=f"{row['avg_rating']:.3f}\n({row['total_ratings']:,} ratings)",
        ha='center',
        va='bottom',
        fontsize=10,
        fontweight='bold',
        color='green'
    )

plt.title('Top 10 Highest-Rated Movies (min 1000 ratings)', 
          fontsize=20, pad=40, fontweight='bold')
plt.xlabel('Movie Title', fontsize=14)
plt.ylabel('Average Rating (1–5)', fontsize=14)
plt.ylim(4.0, 5.0)
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=50, ha='right', fontsize=13)

plt.tight_layout()
plt.show()

# Display table
print("Top 10 Movies (no merge):")
display(top10[['title_display', 'avg_rating', 'total_ratings', 'movie_id']])

In [ ]:
plt.figure(figsize=(14, 9))

top10_Reversed = top10.sort_values('avg_rating', ascending=True)

bars = plt.barh(
    y=top10_Reversed['title_display'],
    width=top10_Reversed['avg_rating'],
    color='#0062ff',
    edgecolor='navy',
    height=0.7,
    alpha=0.95
)

# Add the exact rating + number of ratings on each bar
for bar, row in zip(bars, top10_Reversed.itertuples()):
    plt.text(
        x=bar.get_width() + 0.005,                    # slightly right of bar
        y=bar.get_y() + bar.get_height()/2,           # center vertically
        s=f"{row.avg_rating:.3f}  ({row.total_ratings:,} ratings)",
        va='center',
        ha='left',
        fontsize=11,
        fontweight='bold',
        color='darkblue'
    )

# Styling — same as your scatterplot
plt.title('Top 10 Highest-Rated Movies (min 1000 ratings)', 
          fontsize=20, pad=40, fontweight='bold')
plt.xlabel('Average Rating (1–5)', fontsize=14)
plt.ylabel('Movie Title', fontsize=14)
plt.xlim(4.0, 5.0)
plt.grid(True, axis='x', linestyle='--', alpha=0.7)

# Clean y-axis labels (movie titles)
plt.yticks(fontsize=13)

plt.tight_layout()
plt.show()

 ### Work in progress/Miscellaneous

Giacomo's Ting

In [ ]:
tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';",
    conn
    )['name'].tolist()

print("=== DATA DICTIONARY ===\n")

for table in tables:
    print(f"Table: {table}")
    print("-" * (7 + len(table)))

    # Get actual column info from PRAGMA but filter to nice output
    schema = pd.read_sql(f"PRAGMA table_info('{table}')", conn)

    # Keep only real schema fields you want (remove cid, default, pk if desired)
    clean_schema = schema[['name', 'type', ]]

    print(clean_schema.to_string(index=False))
    print("\n")

### Models

K-means clustering algorithm for movies

In [ ]:
sync_dataframe(merged_data)
#marco
merged = dfs["merged_data"]

# feautures im basing my clusters on
movie_features = [
    "movie_total_ratings",
    "movie_avg_rating",
    "movie_std_rating",
    "movie_min_rating",
    "movie_max_rating",
    "unique_users",
    "year_of_release",
]

# making a table 1 row - 1 movie
movies_for_clustering = (
    merged
    .groupby("movie_id")[movie_features]
    .first()
    .reset_index()
)

# getting all number variables except for movie_id
X = movies_for_clustering[movie_features].astype(float)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# number of clusters
n_clusters = 4

kmeans = KMeans(
    n_clusters=n_clusters,
    random_state=42,
    n_init=10
)

cluster_labels = kmeans.fit_predict(X_scaled)

# adding cluster ids to the table
movies_for_clustering["cluster_id"] = cluster_labels

# saving
add_new_df('movie_clusters', 'movie_clusters', movies_for_clustering)

# X_scaled из предыдущего шага (после StandardScaler)
elbow_df = elbow_method(X_scaled, 1, 20)
print(elbow_df)

# printing how many movies in each cluster, only for debugging
cluster_movies_amount_str = movies_for_clustering["cluster_id"].value_counts().sort_index()
print(f"---movies in each cluster--- \n{cluster_movies_amount_str}")
# Silhouette score
print(compute_silhouette_score(X_scaled, cluster_labels))

K-means clustering algorithm for movies without the outliers (cluster 4)

In [ ]:
# фильмы, которые НЕ входят в маленький кластер
valid_movie_ids = movies_for_clustering[movies_for_clustering["cluster_id"] != 3]["movie_id"]

# новый merged, очищенный от кластера 3
merged_clean = merged[merged["movie_id"].isin(valid_movie_ids)].copy()

movie_features = [
    "movie_total_ratings",
    "movie_avg_rating",
    "movie_std_rating",
    "movie_min_rating",
    "movie_max_rating",
    "unique_users",
    "year_of_release",
]

movies_clean = (
    merged_clean
    .groupby("movie_id")[movie_features]
    .first()
    .reset_index()
)

from sklearn.preprocessing import StandardScaler

X_clean = movies_clean[movie_features].astype(float)

scaler_clean = StandardScaler()
X_clean_scaled = scaler_clean.fit_transform(X_clean)

clean_cluster_labels = kmeans.fit_predict(X_clean_scaled)

kmeans_clean = KMeans(
    n_clusters=4,     # оставляем то же k
    random_state=42,
    n_init=10
)

labels_clean = kmeans_clean.fit_predict(X_clean_scaled)

movies_clean["cluster_id"] = labels_clean

elbow_df = elbow_method(X_clean_scaled, 1, 20)
print(elbow_df)

# printing how many movies in each cluster, only for debugging
clean_cluster_movies_amount_str = movies_clean["cluster_id"].value_counts().sort_index()
print(f"---movies in each cluster--- \n{clean_cluster_movies_amount_str}")
# Silhouette score
print(compute_silhouette_score(X_clean_scaled, clean_cluster_labels))

inspect_cluster(movies_clean, merged_clean, 3)

In [ ]:
#r

In [ ]:
#r

Agglomerative clustering algorithm for movies

In [ ]:
from scipy.cluster.hierarchy import fcluster

sync_dataframe(merged_data)
merged = dfs["merged_data"]

# Features for clustering
movie_features = [
    "movie_total_ratings",
    "movie_avg_rating",
    "movie_std_rating",
    "movie_min_rating",
    "movie_max_rating",
    "unique_users",
    "year_of_release",
]

# One row per movie
movies_for_h_clustering = (
    merged
    .groupby("movie_id")[movie_features]
    .first()
    .reset_index()
)

print(movies_for_h_clustering.head())

# Matrix of fs
X = movies_for_h_clustering[movie_features].astype(float)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Z = linkage(X_scaled, method="ward")

# Building a dendogram
plt.figure(figsize=(12, 6))
dendrogram(
    Z,
    truncate_mode="lastp",
    p=50,
    leaf_rotation=90.,
    leaf_font_size=8.,
    show_contracted=True
)
plt.title("Movie clustering dendrogram (truncated)")
plt.xlabel("Cluster index or movie groups")
plt.ylabel("Distance")
plt.tight_layout()
plt.show()

n_clusters = 4  # number of clusters

h_cluster_labels = fcluster(Z, t=n_clusters, criterion="maxclust")

movies_for_h_clustering["cluster_id"] = h_cluster_labels - 1

# printing how many movies in each cluster, only for debugging
h_cluster_movies_amount_str = movies_for_h_clustering["cluster_id"].value_counts().sort_index()
print(f"---movies in each cluster--- \n{h_cluster_movies_amount_str}")
# Silhouette score
hier_labels = h_clustering_movies["cluster_id"].to_numpy()
print(compute_silhouette_score(X_clean_scaled, hier_labels))

# saving
add_new_df("movie_clusters_h", "movie_clusters_h", movies_for_h_clustering)

In [ ]:
#m

K-means on users

In [ ]:
sync_dataframe(merged_data)
data = merged_data.copy()

user_features = [
    'user_total_ratings',
    'unique_movies',
    'activity_days',
    'user_avg_rating',
    'user_std_rating',
]

# Taking first row per user
users_for_clustering = (
    data.groupby("customer_id")[user_features]
    .first()
    .reset_index()
)

data = users_for_clustering[user_features].astype(float)
data = data[user_features].dropna()

scalar = StandardScaler()
scaled_data = scalar.fit_transform(data)

print(elbow_method(data))

# k-means
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
cluster_labels = kmeans.fit_predict(scaled_data)

users_for_clustering["cluster_id"] = cluster_labels
print(users_for_clustering["cluster_id"].value_counts().sort_index())

add_new_df("user_clusters", "user_clusters", users_for_clustering)

Markdown

In [ ]:
X = user_clusters[user_features].to_numpy()
labels = user_clusters["cluster_id"].to_numpy()


sample_size = 20000   
idx = np.random.choice(len(X), size=sample_size, replace=False)

X_sample = X[idx]
labels_sample = labels[idx]

score = silhouette_score(X_sample, labels_sample)

print("Sampled silhouette score:", score)
for cid in [0, 1, 2]:
    print(f"\nCLUSTER {cid}\n")

    cluster = users_for_clustering[users_for_clustering["cluster_id"] == cid]

    display(cluster[user_features].describe())
    display(cluster.head(20))

In [ ]:
#g

In [ ]:
#g